In [1]:
import os
import math
import imageio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread,imshow
from skimage.measure import label, regionprops

#Set paths accordingly
segmentations_path = "C:\\Users\\SWONG\\PRS_project\\Data_Set\\val\\melanoma\\segmentation\\"
image_input_path = "C:\\Users\\SWONG\\PRS_project\\Data_Set\\val\\melanoma\\images\\"
image_output_path = "C:\\Users\\SWONG\\PRS_project\\Data_Set\\val\\melanoma\\asym_processed\\"

In [2]:
#this function is the same as the one in border_cropping_preprocessing, to get the binary perimeter outline

def get_perimeter(seg, seg_border, width, height):
#getting the image perimeter from the segmentation mask
    
    # getting border pixels in the left to right direction
    for i in range(height):
        for j in range(width - 1):
            try:
                if seg[i][j] == 0 and seg[i][j + 1] == 1:
                    seg_border[i][j] = 1
                elif seg[i][j] == 1 and seg[i][j + 1] == 0:
                    seg_border[i][j + 1] = 1
            except:
                print('Pixel out of range')
                
    # getting border pixels in the right to left direction
    for i in range(height):
        for j in range(width - 1):
            try:
                if seg[i][width - j - 1] == 0 and seg[i][width - j - 2] == 1:
                    seg_border[i][width - j - 1] = 1
                elif seg[i][width - j - 1] == 1 and seg[i][width - j - 2] == 0:
                    seg_border[i][width - j - 2] = 1
            except:
                print('Pixel out of range')

    # getting border pixels in the up to down direction
    for j in range(width):
        for i in range(height - 1):
            try:
                if seg[i][j] == 0 and seg[i + 1][j] == 1:
                    seg_border[i][j] = 1
                elif seg[i][j] == 1 and seg[i + 1][j] == 0:
                    seg_border[i + 1][j] = 1
            except:
                print('Pixel out of range')

    # getting border pixels in the down to up direction
    for j in range(width):
        for i in range(height - 1):
            try:
                if seg[height - i - 1][j] == 0 and seg[height - i - 2][j] == 1:
                    seg_border[height - i - 1][j] = 1
                elif seg[height - i - 1][j] == 1 and seg[height - i - 2][j] == 0:
                    seg_border[height - i - 2][j] = 1
            except:
                print('Pixel out of range')
                
    return seg_border

In [3]:
#get the image perimeter pixel values as a list with arrays of length 3
def get_px_values(img, perimeter):
    #this is a list that will contain arrays of length 3
    perimeter_px_values = []
    
    height = img.shape[0]
    width = img.shape[1]
    
    for i in range(height):
        for j in range(width):
            if perimeter[i][j] == 1:
                perimeter_px_values.append(img[i][j])
    
    return perimeter_px_values

In [4]:
#calculate the average perimeter pixel value
def avg_pixel_values(perimeter_px_values):
    length = len(perimeter_px_values)
    sum_red = 0
    sum_green = 0
    sum_blue = 0
    
    for i in perimeter_px_values:
        sum_red = sum_red + i[0]
        sum_green = sum_green + i[1]
        sum_blue = sum_blue + i[2]
    return sum_red/length, sum_green/length, sum_blue/length

In [5]:
#step 1 of the asymmetry preprocessing: replace non-lesion pixels(outside the seg mask) with the avg perimeter colour
def asym_step_one(img, seg, avg_red, avg_green, avg_blue):
    step_1_img = np.empty(img.shape, dtype=np.uint8)
    
    height = img.shape[0]
    width = img.shape[1]
    
    for i in range(height):
        for j in range(width):
            if seg[i][j] == 0:
                step_1_img[i][j][0] = avg_red
                step_1_img[i][j][1] = avg_green
                step_1_img[i][j][2] = avg_blue
            else:
                step_1_img[i][j][:] = img[i][j][:]
          
    return step_1_img

In [6]:
#gets the eqn parameters of major and minor axes of segmentation mask, using tangent and orientation angle(in radian)
def get_axes_eqns(orientation, centroid_0, centroid_1):
    
    gradient_mj = math.tan((math.pi/2)-orientation)
    intercept_mj = centroid_0 - gradient_mj * centroid_1
    
    gradient_mn = 1/(math.tan((math.pi/2)+orientation))
    intercept_mn = centroid_0 - gradient_mn * centroid_1
    
    return gradient_mj, gradient_mn, intercept_mj, intercept_mn

In [7]:
#gets the pixel difference of the reflection over both major and minor axes
def get_px_diff(img, gradient_mj, intercept_mj, gradient_mn, intercept_mn):
        
    height = img.shape[0]
    width = img.shape[1]
    
    #create empty 3d arrays to store the difference values
    diff_arr_mj = np.empty(img.shape, dtype=np.uint8)
    diff_arr_mn = np.empty(img.shape, dtype=np.uint8)
    diff_arr_mjmn = np.empty(img.shape, dtype=np.uint8)
    
    #looping through each pixel in the image 
    for i in range(height):
        for j in range(width):
            #in case the gradient of the axis is zero
            if gradient_mj !=0 and gradient_mn !=0:
                x_dist_from_mj_axis = int(j - ((i-intercept_mj)/gradient_mj))
                y_dist_from_mj_axis = int(i - (gradient_mj * j + intercept_mj)) 
                x_dist_from_mn_axis = int(j - ((i-intercept_mn)/gradient_mn))
                y_dist_from_mn_axis = int(i - (gradient_mn * j + intercept_mn))
                reflected_y_dist_from_mj_axis = x_dist_from_mj_axis
                reflected_x_dist_from_mj_axis = y_dist_from_mj_axis
                reflected_y_dist_from_mn_axis = x_dist_from_mn_axis
                reflected_x_dist_from_mn_axis = y_dist_from_mn_axis
                reflected_x_coordinate_mj = j + reflected_x_dist_from_mj_axis
                reflected_y_coordinate_mj = i + reflected_y_dist_from_mj_axis
                reflected_x_coordinate_mn = j + reflected_x_dist_from_mn_axis
                reflected_y_coordinate_mn = i + reflected_y_dist_from_mn_axis            
                reflected_x_coordinate_mjmn = j + reflected_x_dist_from_mj_axis + reflected_x_dist_from_mn_axis
                reflected_y_coordinate_mjmn = i + reflected_y_dist_from_mj_axis + reflected_y_dist_from_mn_axis
            elif gradient_mj == 0:
                reflected_x_coordinate_mj = j
                y_dist_from_mj_axis = int(i - intercept_mj)
                reflected_y_coordinate_mj = i - 2*y_dist_from_mj_axis
                reflected_y_coordinate_mn = i
                x_dist_from_mn_axis = int(j - intercept_mn)
                reflected_x_coordinate_mn = j - 2*x_dist_from_mn_axis
                reflected_x_coordinate_mjmn = reflected_x_coordinate_mn
                reflected_y_coordinate_mjmn = reflected_y_coordinate_mj
            #else if gradient_mn==0
            else:
                reflected_x_coordinate_mn = j
                y_dist_from_mn_axis = int(i - intercept_mn)
                reflected_y_coordinate_mn = i - 2*y_dist_from_mn_axis
                reflected_y_coordinate_mj = i
                x_dist_from_mj_axis = int(j - intercept_mj)
                reflected_x_coordinate_mj = j - 2*x_dist_from_mj_axis
                reflected_x_coordinate_mjmn = reflected_x_coordinate_mj
                reflected_y_coordinate_mjmn = reflected_y_coordinate_mn
            
            if 0 <= reflected_x_coordinate_mj < width and 0 <= reflected_y_coordinate_mj < height:
                diff_arr_mj[i][j][:] = img[i][j][:] - img[reflected_y_coordinate_mj][reflected_x_coordinate_mj][:]
            else:
                diff_arr_mj[i][j][:] = 0
                
            if 0 <= reflected_x_coordinate_mn < width and 0 <= reflected_y_coordinate_mn < height:
                diff_arr_mn[i][j][:] = img[i][j][:] - img[reflected_y_coordinate_mn][reflected_x_coordinate_mn][:]
            else:
                diff_arr_mn[i][j][:] = 0     
                
            if 0 <= reflected_x_coordinate_mjmn < width and 0 <= reflected_y_coordinate_mjmn < height:
                diff_arr_mjmn[i][j][:] = img[i][j][:] - img[reflected_y_coordinate_mjmn][reflected_x_coordinate_mjmn][:]
            else:
                diff_arr_mjmn[i][j][:] = 0
                
    return diff_arr_mj, diff_arr_mn, diff_arr_mjmn

In [8]:
img_list = os.listdir(image_input_path)

for img_name in img_list:

    #read the segmentation file
    seg_name = img_name.split('.')[0]+"_segmentation.png"
    seg = imread(segmentations_path + seg_name, as_gray=True)

    #preprocessing the seg array to make it binary
    seg = seg/255

    #get the perimeter outline of the lesion in a 2d binary array
    height = seg.shape[0]
    width = seg.shape[1]
    seg_border = np.empty(seg.shape, dtype=np.uint8)
    perimeter = get_perimeter(seg, seg_border, width, height)
    
    #read the image file
    img = imread(image_input_path+img_name, as_gray=False)

    #perimeter_px_values is a list containing arrays of length 3
    #use perimeter outline to get the perimeter_px_values from original image
    perimeter_px_values = get_px_values(img, perimeter)
    
    #get the average value of perimeter pixels
    avg_red, avg_green, avg_blue = avg_pixel_values(perimeter_px_values)
    
    #replace the non-lesion pixels with the average RGB values of the lesion perimeter
    step_1_img = asym_step_one(img, seg, avg_red, avg_green, avg_blue)

    #using skimage regionprops to get orientation, centroid_0, centroid_1, to calculate the axes eqn parameters
    label_seg = label(seg)
    props = regionprops(label_seg)
    gradient_mj, gradient_mn, intercept_mj, intercept_mn = get_axes_eqns(props[0].orientation, 
                                                                         props[0].centroid[0], 
                                                                         props[0].centroid[1])
    
    
    #get the 3 set of difference values
    diff_mj, diff_mn, diff_mjmn = get_px_diff(step_1_img, gradient_mj, intercept_mj, gradient_mn, intercept_mn)

    #calculate the average for each channel in each pixel
    diff_avg = np.empty(img.shape,dtype=np.uint8)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            diff_avg[i][j][:] = (diff_mj[i][j][:] + diff_mn[i][j][:] + diff_mjmn[i][j][:])/3

    #save the diff_avg as an image to output path
    imageio.imwrite(image_output_path + img_name, diff_avg)   